In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
calls = pd.read_csv('data/CALLS_CLIMATE.csv')

In [4]:
kospi_downloaded = pd.read_pickle('history/kospi_download_result.pkl')
kospi = pd.read_csv('metadata/kospi.csv', error_bad_lines=False)

b'Skipping line 437: expected 12 fields, saw 13\n'


In [5]:
kospi[0:2]

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
0,1,095570,AJ네트웍스,147603.0,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)",789.0
1,2,068400,AJ렌터카,147601.0,운송장비 임대업,"22,146,300","11,073,150,000",500,원(KRW),1544-1600,서울특별시 구로구 서부샛길 822,789.0


In [6]:
kospi_downloaded.head()

,종목코드,상장일,결과,기업명
0,095570,2015-08-21,Naver,AJ네트웍스
1,068400,2012-07-27,Naver,AJ렌터카
2,006840,1999-08-11,Naver,AK홀딩스
3,027410,2014-05-19,Naver,BGF
4,282330,2017-12-08,Naver,BGF리테일


In [7]:
def get_company_name(ticker) : 
    return kospi[kospi['종목코드']==ticker]['기업명'].values[0]

kospi_downloaded['기업명'] = kospi_downloaded['종목코드'].apply(lambda x : get_company_name(x))
kospi_downloaded.to_pickle('history/kospi_download_result.pkl')

# Show by ticker

In [8]:
tickers = kospi_downloaded['종목코드']
tickers

0      095570
1      068400
2      006840
3      027410
4      282330
        ...  
783    079980
784    005010
785    069260
786    000540
787    003280
Name: 종목코드, Length: 788, dtype: object

In [24]:
@interact
def show_by_ticker(ticker = tickers) : 
    metadata = kospi[kospi['종목코드'] == ticker]
    display(metadata)
    
    data = pd.read_pickle('data/kospi/%s.data'%ticker)
    data.index = pd.to_datetime(data.index)
    data['Close'] = pd.to_numeric(data['Close'])
    data['Volume'] = pd.to_numeric(data['Volume'])
    display(data.head())
    
    fig = plt.figure(figsize=(10, 6))
    
    
    ax1 = fig.add_subplot(111)
    lns1 = ax1.plot(data.index, data['Close'], label='Close')
    plt.xticks(rotation=90)
    ax2 = ax1.twinx()
    lns2 = ax2.plot(data.index, data['Volume'], color='orange', alpha=0.4, label='Volume')
    
    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    plt.legend(lns, labs, loc=0)
    plt.title('ticker : %s, %s ~ %s' % (ticker, data.index[0], data.index[-1]))
    
    plt.show()

interactive(children=(Dropdown(description='ticker', options=('095570', '068400', '006840', '027410', '282330'…

# Show by company name

In [25]:
company_names = kospi_downloaded['기업명']
company_names

0      AJ네트웍스
1       AJ렌터카
2       AK홀딩스
3         BGF
4      BGF리테일
        ...  
783       휴비스
784       휴스틸
785       휴켐스
786      흥국화재
787      흥아해운
Name: 기업명, Length: 788, dtype: object

In [26]:
@interact
def show_by_company_name(company_name = company_names) : 
    metadata = kospi[kospi['기업명'] == company_name]
    display(metadata)
    
    data = pd.read_pickle('data/kospi/%s.data'%ticker)
    data.index = pd.to_datetime(data.index)
    data['Close'] = pd.to_numeric(data['Close'])
    data['Volume'] = pd.to_numeric(data['Volume'])
    display(data.head())
    
    fig = plt.figure(figsize=(10, 6))
    
    
    ax1 = fig.add_subplot(111)
    lns1 = ax1.plot(data.index, data['Close'], label='Close')
    plt.xticks(rotation=90)
    ax2 = ax1.twinx()
    lns2 = ax2.plot(data.index, data['Volume'], color='orange', alpha=0.4, label='Volume')
    
    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    plt.legend(lns, labs, loc=0)
    plt.title('ticker : %s, %s ~ %s' % (ticker, data.index[0], data.index[-1]))
    
    plt.show()

interactive(children=(Dropdown(description='company_name', options=('AJ네트웍스', 'AJ렌터카', 'AK홀딩스', 'BGF', 'BGF리테일…